In [1]:
!pip install tensorflow opencv-python-headless face_recognition

import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image
import face_recognition
from datetime import datetime
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=05f4a79caacdd0eefc3d0805d46a2f811a89681436a943d06bbe6feb8370c1c5
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models
Mounted at /content/drive


In [2]:
directories = [
    "/content/drive/MyDrive/Reconhecimento_Facial/cadastrados",
    "/content/drive/MyDrive/Reconhecimento_Facial/frequência"
]

for directory in directories:
    if not os.path.exists(directory):
        os.makedirs(directory)

In [3]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import IPython.display as display
from PIL import Image
import io

def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
    async function takePhoto(quality) {
        const div = document.createElement('div');
        const capture = document.createElement('button');
        capture.textContent = 'Capture';
        div.appendChild(capture);

        const video = document.createElement('video');
        video.style.display = 'block';
        const stream = await navigator.mediaDevices.getUserMedia({video: true});

        document.body.appendChild(div);
        div.appendChild(video);
        video.srcObject = stream;
        await video.play();

        // Resize the output to match the aspect ratio of the video.
        google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

        // Wait for capture to be clicked.
        await new Promise((resolve) => capture.onclick = resolve);

        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        stream.getTracks().forEach(track => track.stop());
        div.remove();

        const dataUrl = canvas.toDataURL('image/jpeg', quality);
        return dataUrl;
    }
    ''')
    display.display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename


In [35]:
import os
from datetime import datetime
import numpy as np
from PIL import Image
import face_recognition

cadastrados_path = "/content/drive/MyDrive/Reconhecimento_Facial/cadastrados"
frequency_path = "/content/drive/MyDrive/Reconhecimento_Facial/frequência"

def add_new_person(user_id, user_name):
    image_path = take_photo(filename=os.path.join(cadastrados_path, f"{user_id}_{user_name}.jpg"))
    print(f"Imagem capturada e salva em {image_path}")

def get_user_name_and_id():
    user_data = {}
    for filename in os.listdir(cadastrados_path):
        if filename.endswith(".jpg"):
            user_id, user_name = filename.split('_', 1)
            user_name = user_name.replace('.jpg', '')
            user_data[int(user_id)] = user_name
    return user_data

def log_attendance(face_id, user_name):
    now = datetime.now()
    current_time = now.strftime("%Y-%m-%d %H:%M:%S")

    log_file = os.path.join(frequency_path, f"{face_id}.txt")
    if os.path.exists(log_file):
        with open(log_file, "a") as f:
            f.write(f"Saída: {current_time} - {user_name}\n")
    else:
        with open(log_file, "w") as f:
            f.write(f"Entrada: {current_time} - {user_name}\n")

def recognize_faces_and_log():
    captured_image_path = take_photo(filename='captured_image.jpg')
    image = face_recognition.load_image_file(captured_image_path)
    face_locations = face_recognition.face_locations(image)
    if face_locations:
        user_data = get_user_name_and_id()
        for face_location in face_locations:
            top, right, bottom, left = face_location
            face_image = image[top:bottom, left:right]
            pil_image = Image.fromarray(face_image).resize((224, 224))
            face_array = np.array(pil_image) / 255.0
            face_array = np.expand_dims(face_array, axis=0)

            predictions = model.predict(face_array)
            face_id = np.argmax(predictions)

            confidence = np.max(predictions)
            if confidence > 0.75:
                user_name = user_data.get(face_id, "Unknown")
                log_attendance(face_id, user_name)
                print(f"Presença confirmada para {user_name}")
            else:
                print("Rosto não reconhecido")
    else:
        print("Nenhuma face encontrada na imagem capturada.")


In [5]:
model_save_path = '/content/drive/MyDrive/Reconhecimento_Facial/saved_model_3'
model = load_model(model_save_path)

In [ ]:
def menu():
    while True:
        print("Sistema de Frequência por Reconhecimento Facial")
        print("1. Adicionar nova pessoa")
        print("2. Verificar presença")
        print("3. Sair")

        choice = input("Escolha uma opção: ")

        if choice == "1":
            user_id = input("Digite o ID da pessoa: ")
            user_name = input("Digite o nome da pessoa: ")
            add_new_person(user_id, user_name)
        elif choice == "2":
            recognize_faces_and_log()
        elif choice == "3":
            print("Saindo do sistema...")
            break
        else:
            print("Opção inválida. Tente novamente.")

menu()